In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

DB_USER = os.getenv("POSTGRES_USER")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB_HOST = os.getenv("POSTGRES_HOST")
DB_PORT = os.getenv("POSTGRES_PORT")
DB_NAME = os.getenv("POSTGRES_DB")
SCHEMA = os.getenv("POSTGRES_SCHEMA")

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

def read_table(table_name: str):
    query = f'SELECT * FROM {SCHEMA}.{table_name};'
    return pd.read_sql(query, engine)

print(read_table('users').head())

   user_id signup_date       region
0        1  2024-04-12       Казань
1        2  2024-03-15       Москва
2        3  2024-02-22  Новосибирск
3        4  2024-02-27       Казань
4        5  2024-01-15       Москва
